Load Data

In [63]:
# todo: make available for cloud
import json
import os

# Load Kaggle API token
with open("/Users/simi/Downloads/kaggle.json") as f:
    api_token = json.load(f)

# Define environment variables (Kaggle API client expects these)
os.environ['KAGGLE_USERNAME'] = api_token['username']
os.environ['KAGGLE_KEY'] = api_token['key']


In [64]:
from kaggle.api.kaggle_api_extended import KaggleApi
import time

# Initialize the API
api = KaggleApi()
api.authenticate()

# Define the dataset path
dataset_path = "jiashenliu/515k-hotel-reviews-data-in-europe"

# Download all files of the dataset
start = time.time()
api.dataset_download_files(dataset_path, path="data", unzip=True)
end = time.time()
print(f"Time to download and extract: {round(end - start, 2)} seconds")


Time to download and extract: 2.61 seconds


In [65]:
import pandas as pd

# Load the hotel reviews from CSV
start = time.time()
df = pd.read_csv('data/Hotel_Reviews.csv') # download from kaggle
end = time.time()
print(f"Loading took {round(end - start, 2)} seconds")

Loading took 1.71 seconds


Cleaning Data

In [66]:
# check for missing values
df.isna().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [67]:
# We will drop the null rows because we can't replace the lat or lng with mean or median value, that will change the right information hotel adderss
df.dropna(inplace=True,axis=0)

In [68]:
print("Duplicated rows before: ",df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Duplicated rows after: ",df.duplicated().sum())

Duplicated rows before:  526
Duplicated rows after:  0


In [69]:
# Replace all the addresses with a shortened, more useful form
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 
    else:
        return row.Hotel_Address
    

df["Hotel_Address"] = df.apply(replace_address, axis = 1)

In [70]:
# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"], axis = 1, inplace=True)
# Replace `Total_Number_of_Reviews` and `Average_Score` with own calculated values
df["Total_Number_of_Reviews"] = df.groupby('Hotel_Name')['Hotel_Name'].transform('count')
df["Average_Score"] = df.groupby('Hotel_Name')['Reviewer_Score'].transform('mean').round(1)

In [71]:
# We want to find the most useful tags to keep
# Remove opening and closing brackets and all quotes too, split the strings into a list
tag_list_df = df.Tags.str.strip("[']").str.replace(" ', '", ",", regex=False).str.split(',', expand=True)

# Remove leading and trailing spaces from each column and assign them back to the dataframe
for i in range(6):
    df[f"Tag_{i+1}"] = tag_list_df[i].str.strip()

# Merge the 6 columns into one with melt
df_tags = df.melt(value_vars=[f"Tag_{i+1}" for i in range(6)])

# Print the shape of the tags with no filtering
print("The shape of the tags with no filtering:", df_tags.shape)

# Filter the tags, get the value counts
df_tags = df_tags[~df_tags.value.str.contains("Standard|room|Stayed|device|Beds|Suite|Studio|King|Superior|Double", na=False, case=False)]
tag_vc = df_tags.value.value_counts().reset_index(name="count").query("count > 1000")

# Print the top 10 (there should only be 9 and we'll use these in the filtering section)
print(tag_vc[:10])

The shape of the tags with no filtering: (3071664, 2)
                        value   count
0                Leisure trip  414707
1                      Couple  250467
2               Solo traveler  107730
3               Business trip   82341
4                       Group   64890
5  Family with young children   60563
6  Family with older children   26167
7      Travelers with friends    2127
8                  With a pet    1385


In [72]:
# Process the Tags into new columns
for tag in tag_vc["value"]:
    df[tag] = df.Tags.apply(lambda x: 1 if tag in x else 0)

In [73]:
# No longer need any of these columns
df.drop(["Review_Date", "Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "days_since_review", 
         "Total_Number_of_Reviews_Reviewer_Has_Given", "Tags", "Tag_1", "Tag_2", "Tag_3", "Tag_4", "Tag_5", "Tag_6"], axis = 1, inplace=True)

In [74]:
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score,lat,lng,Leisure trip,Couple,Solo traveler,Business trip,Group,Family with young children,Family with older children,Travelers with friends,With a pet
0,"Amsterdam, Netherlands",7.8,Hotel Arena,Russia,I am so angry that i made this post available...,405,Only the park outside of the hotel was beauti...,2.9,52.360576,4.915968,1,1,0,0,0,0,0,0,0
1,"Amsterdam, Netherlands",7.8,Hotel Arena,Ireland,No Negative,405,No real complaints the hotel was great great ...,7.5,52.360576,4.915968,1,1,0,0,0,0,0,0,0
2,"Amsterdam, Netherlands",7.8,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,405,Location was good and staff were ok It is cut...,7.1,52.360576,4.915968,1,0,0,0,0,1,0,0,0
3,"Amsterdam, Netherlands",7.8,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,405,Great location in nice surroundings the bar a...,3.8,52.360576,4.915968,1,0,1,0,0,0,0,0,0
4,"Amsterdam, Netherlands",7.8,Hotel Arena,New Zealand,You When I booked with your company on line y...,405,Amazing location and building Romantic setting,6.7,52.360576,4.915968,1,1,0,0,0,0,0,0,0


EDA (Exploratory Data Analysis) 

In [75]:
# Since Lea and I also can't decide which city to choose the city of love is a good start for some romantic action but also fot the analysis
paris_df = df[df["Hotel_Address"] == "Paris, France"]
paris_df=paris_df.drop_duplicates(subset='Hotel_Name')
paris_df.shape

(455, 19)

In [76]:
import folium
from tqdm import tqdm

def assign_color(score: float) -> str:
    if score >= 9.3:
        return 'green'
    elif score >= 8.3:
        return 'lightgreen'
    elif score >= 7.3:
        return 'orange'
    else:
        return 'red'

# Initialize map with starting location
paris_map = folium.Map(location=[paris_df['lat'].mean(), paris_df['lng'].mean()], zoom_start=12)

for idx, row in tqdm(iterable=paris_df.iterrows(), total=paris_df.shape[0]):
    folium.Marker(
        location=[row['lat'], row['lng']], 
        popup=f'{row["Hotel_Name"]}: {row["Average_Score"]}',
        icon=folium.Icon(color=assign_color(row["Average_Score"]), icon="info-sign")
    ).add_to(paris_map)

# Show the map
paris_map


100%|██████████| 455/455 [00:00<00:00, 23724.91it/s]


Analysis

In [77]:
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/simi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/simi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
vader_sentiment = SentimentIntensityAnalyzer()
cache = set(stopwords.words("english"))

# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review: str) -> float:    
    if review == "No Negative" or review == "No Positive":
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]

def remove_stopwords(review: str) -> str:
    text = " ".join([word for word in review.split() if word not in cache])
    return text

In [79]:
# Remove stop words - can be slow for a lot of text!
# Ryan Han (ryanxjhan on Kaggle) has a great post measuring performance of different stop words removal approaches
# https://www.kaggle.com/ryanxjhan/fast-stop-words-removal # using the approach that Ryan recommends
start = time.time()

# Remove the stop words from both columns
df.Negative_Review = df.Negative_Review.apply(remove_stopwords)   
df.Positive_Review = df.Positive_Review.apply(remove_stopwords)
end = time.time()
print("Removing stop words took " + str(round(end - start, 2)) + " seconds")

Removing stop words took 1.64 seconds


In [80]:
# Add a negative sentiment and positive sentiment column
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews
Calculating sentiment took 56.42 seconds


In [81]:
df = df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(df[["Negative_Review", "Negative_Sentiment"]])
df = df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(df[["Positive_Review", "Positive_Sentiment"]])

                                          Negative_Review  Negative_Sentiment
186584  So bad experience memories I hotel The first n...             -0.9920
129503  First charged twice room booked booking second...             -0.9896
307286  The staff Had bad experience even booking Janu...             -0.9889
201953  Everything DO NOT STAY AT THIS HOTEL I never i...             -0.9886
452092  No WLAN room Incredibly rude restaurant staff ...             -0.9884
...                                                   ...                 ...
138365  Wifi terribly slow I speed test network upload...              0.9938
79215   I find anything hotel first I walked past hote...              0.9938
278506  The property great location There bakery next ...              0.9945
339189  Guys I like hotel I wish return next year Howe...              0.9948
480509  I travel lot far visited countless number hote...              0.9957

[511944 rows x 2 columns]
                                     

In [82]:
# Reorder the columns (This is cosmetic, but to make it easier to explore the data later)
df = df.reindex(["Hotel_Name", "Hotel_Address", "Total_Number_of_Reviews", "Average_Score", "Reviewer_Score", "Negative_Sentiment", "Positive_Sentiment", "Reviewer_Nationality", "Leisure_trip", "Couple", "Solo_traveler", "Business_trip", "Group", "Family_with_young_children", "Family_with_older_children", "With_a_pet", "Negative_Review", "Positive_Review"], axis=1)